# Projet maison n° 4

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# imports
import requests
import re
import json
from bs4 import BeautifulSoup
from multiprocessing import Pool

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [ ]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"


def get_prices_from_url(url):
    prices = {}

    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    nb_of_tables = len(list(soup.find_all("div", class_="pricing-table-header")))
    for i in range(nb_of_tables):
        # PRICE AND TITLE
        pricing_tables = soup.find_all("div", class_="pricing-table-header")
        title = pricing_tables[i].find("h2").text
        price = pricing_tables[i].find("span").text
        price = re.search("\$\d+", price).group(0)

        # FILE_STORAGE
        pricing_table_list = soup.find_all("ul", class_="pricing-table-list")
        file_storage = pricing_table_list[i].find_all("li")[3].text
        file_storage = re.search("\d+[A-Z]{1,2}", file_storage).group(0)

        # DATABASE
        database = pricing_table_list[i].find_all("li")[4].text
        database = re.search("\d+", database).group(0)

        prices[title] = {
            "price": price,
            "storage": file_storage,
            "databases": int(database),
        }

    return prices

prices = get_prices_from_url(url=URL_PAGE3)
print(prices)

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [6]:
# partie B-1
def extract_beer_infos(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    beer_infos_tag = soup.find("div", attrs={"class": "small-12 content-column"})

    beer_infos = {
        "Nom": beer_infos_tag.find("h1").text.strip(),
        "Style": beer_infos_tag.find("dd", class_="small-6 medium-9 columns")
        .find("a")
        .text.strip(),
        "Contenu": int(
            beer_infos_tag.find("dd", class_="small-6 medium-9 columns js-beer-volume")
            .text.strip()
            .split()[0]
        ),
        "Degré d’alcool": float(
            beer_infos_tag.find("dd", class_="small-6 medium-9 columns")
            .find_next("dd", class_="small-6 medium-9 columns")
            .text.strip()
            .replace(",", ".")
            .replace("%", "")
        ),
        "Origine": beer_infos_tag.find(
            "dd", class_="small-6 medium-9 columns js-beer-country"
        ).text.strip(),
        "Brasseur": beer_infos_tag.find("dd", class_="small-6 medium-9 columns")
        .find_next("dd", class_="small-6 medium-9 columns")
        .find_next("dd", class_="small-6 medium-9 columns")
        .text.strip(),
    }

    return beer_infos


beer_infos = extract_beer_infos(
    url="https://www.beerwulf.com/fr-fr/p/bieres/desperados-fut-2l"
)
beer_infos

{'Nom': 'Desperados - Fût 2L The SUB',
 'Style': 'Lager',
 'Contenu': 2,
 'Degré d’alcool': 5.9,
 'Origine': 'France',
 'Brasseur': 'Desperados'}

In [7]:
##### partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"


def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    data = requests.get(url).json()

    # Sequential version (slow):
    for item in data.get("items"):
        beer_info = extract_beer_infos(
            "https://www.beerwulf.com" + item.get("contentReference")
        )
        beer_pages.append(beer_info)

    # Facultatif:
    # Parallel version (faster):

    return beer_pages


beer_list_infos = extract_beer_list_infos(url=URL_BEERLIST_FRANCE)
beer_list_infos

[{'Nom': 'La Cristal IPA du Mont Blanc',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Mont-Blanc'},
 {'Nom': 'Ninkasi Pale Ale',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'Ninkasi Fabriques'},
 {'Nom': 'Desperados',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Desperados'},
 {'Nom': 'Pietra',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'Brasserie Pietra'}]

In [8]:
import unittest


class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn("Personal", prices)
        self.assertIn("Small Business", prices)
        self.assertIn("Enterprise", prices)

        personal = prices["Personal"]
        self.assertIn("price", personal)
        self.assertIn("storage", personal)
        self.assertIn("databases", personal)
        self.assertEqual(personal["price"], "$5")
        self.assertEqual(personal["storage"], "1GB")
        self.assertEqual(personal["databases"], 1)

    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices["Privilege"], {"databases": 100, "price": "$99", "storage": "1TB"}
        )

    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer["Nom"], str)
            self.assertIsInstance(beer["Style"], str)
            self.assertIsInstance(beer["Contenu"], int)
            self.assertIsInstance(beer["Degré d’alcool"], float)
            self.assertEqual(beer["Origine"], "France")
            self.assertIsInstance(beer["Brasseur"], str)


def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [9]:
if __name__ == "__main__":
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 11.482s

OK
